In [7]:
using CSV, DataFrames, Statistics, FreqTables, LinearAlgebra

In [8]:
a1=DataFrame(CSV.File("./csv/asts.csv"));

In [9]:
b1=DataFrame(CSV.File("./csv/philinst.csv")); # comparing to output of instructor.pl

In [10]:
m1=DataFrame(CSV.File("./csv/marks.csv"));

In [11]:
a2=a1[a1.course .== 514,:]; # filter for course id 326

In [12]:
d1=Dict{String, Float64}() # associate ast with cont
for i in 1:length(a2[:,:ast])
    merge!(d1,Dict(a2[i,:ast]=>a2[i,:cont]))
end

In [23]:
m2=DataFrame() # filter for marks relevant to course id
dup=[]
for i in 1:length(a2[:,:ast])
    m2=vcat(m2,m1[m1.ast .== a2[i,:ast],:])
    dup=vcat(dup,sum(nonunique(m1[m1.ast .== a2[i,:ast],:],:sdc)))
end

In [14]:
dup # these should all be zero or else there is duplication

23-element Vector{Any}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [15]:
function ast2cont(x)
    d1[x]
end

ast2cont (generic function with 1 method)

In [16]:
m3=transform(m2,:ast => ByRow(ast2cont) => :cont);

In [17]:
m3g=groupby(m3,:sdc);

In [18]:
b2=combine(m3g,[:mark,:cont] => ((m,c) -> dot(m,c)) => :totalj);

In [19]:
# next steps: (1) account for missing data (2) replace sdc by name and sno (3) sort and export to csv

In [20]:
b3=innerjoin(b1,b2,on=:sdc,makeunique=true);

In [26]:
function rw(x)
    trunc(Int,x+0.5)
end

rw (generic function with 1 method)

In [27]:
b4=sort(select(b3,:sno,:sdc,:fn,:ln,:total,:totalj => ByRow(rw) => :juliatotal),:juliatotal,rev=true)

,sno,sdc,fn,ln,total,juliatotal
,Int64,Int64,String15,String15,Float64,Int64
1,38426516,854333,Anthony,Kohan,0.92,92
2,22213623,507497,Hui,Wong,0.9,90
3,81789018,573322,Anila,Lacroix,0.9,90
4,95078424,605009,Gal,Tabenkin,0.9,90
5,77049898,685939,Colby,Chambers,0.9,90
6,39908355,300556,Nick,Halme,0.89,89
7,40747941,377532,Nicholas,Zurkovic,0.89,89
8,44818177,488081,Jiaxin,Zhang,0.89,89
9,30161814,734042,Annika,Coutinho,0.88,88


In [29]:
b5=select(b4,:sno,:juliatotal)

,sno,juliatotal
,Int64,Int64
1,38426516,92
2,22213623,90
3,81789018,90
4,95078424,90
5,77049898,90
6,39908355,89
7,40747941,89
8,44818177,89
9,30161814,88
